In [1]:
import os
import sys
import tarfile
import pickle
import matplotlib.pyplot as plt
import numpy as np, h5py 
import pandas as pd
from PIL import Image
import tensorflow as tf

pickle_file = 'SVHN.pickle'

In [2]:
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    X_train = save['train_dataset']
    y_train = save['train_labels']
    X_val = save['valid_dataset']
    y_val = save['valid_labels']
    X_test = save['test_dataset']
    y_test = save['test_labels']
    del save  
    print('Training data shape:', X_train.shape)
    print('Training label shape:',y_train.shape)
    print('Validation data shape:', X_val.shape)
    print('Validation label shape:', y_val.shape)
    print('Test data shape:', X_test.shape)
    print('Test label shape:', y_test.shape)

('Training data shape:', (230070, 32, 32, 1))
('Training label shape:', (230070, 6))
('Validation data shape:', (5684, 32, 32, 1))
('Validation label shape:', (5684, 6))
('Test data shape:', (13068, 32, 32, 1))
('Test label shape:', (13068, 6))


In [3]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels)/ predictions.shape[1] / predictions.shape[0])

In [4]:
image_size = 32
num_channels = 1
batch_size = 16
num_labels = 11
patch_size = 5
depth_1 = 16
depth_2 = depth_1 * 2
depth_3 = depth_2 * 3
num_hidden = 64
shape = [batch_size, image_size, image_size, num_channels]
graph = tf.Graph()

In [5]:
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.int32, shape=(batch_size, 6))
    tf_valid_dataset = tf.constant(X_val)
    tf_test_dataset = tf.constant(X_test)
    def init_weights(shape, name):
        return tf.Variable(tf.random_normal(shape=shape, stddev=0.01),name=name)

    def init_biases(shape, name):
        return tf.Variable(tf.constant(1.0, shape=shape),name=name)

    def output_size_pool(input_size, conv_filter_size, pool_filter_size, padding, conv_stride, pool_stride):
        if padding == 'same':
            padding = -1.00
        elif padding == 'valid':
            padding = 0.00
        else:
            return None
        output_1 = (((input_size - conv_filter_size - 2 * padding) / conv_stride) + 1.00)
        output_2 = (((output_1 - pool_filter_size - 2 * padding) / pool_stride) + 1.00)
        output_3 = (((output_2 - conv_filter_size - 2 * padding) / conv_stride) + 1.00)
        output_4 = (((output_3 - pool_filter_size - 2 * padding) / pool_stride) + 1.00)
        output_5 = (((output_4 - conv_filter_size - 2 * padding) / conv_stride) + 1.00)
        return int(output_5)

    w_c1 = init_weights([patch_size, patch_size, num_channels, depth_1], 'w_c1')
    b_c1 = init_biases([depth_1], 'b_c1')
    w_c2 = init_weights([patch_size, patch_size, depth_1, depth_2], 'w_c2')
    b_c2 = init_biases([depth_2], 'b_c2')
    w_c3 = init_weights([patch_size, patch_size, depth_2, depth_3], 'w_c3')
    b_c3 = init_biases([depth_3], 'b_c3')
    final_image_size = output_size_pool(input_size=image_size,conv_filter_size=5, pool_filter_size=2,padding='valid', conv_stride=1,pool_stride=2)
    w_fc1 = init_weights([final_image_size*final_image_size*depth_3, num_hidden], 'w_fc1')
    b_fc1 = init_biases([num_hidden], 'b_fc1')
    w_s1 = init_weights([num_hidden, num_labels], 'w_s1')
    b_s1 = init_biases([num_labels], 'b_s1')
    w_s2 = init_weights([num_hidden, num_labels], 'w_s2')
    b_s2 = init_biases([num_labels], 'b_s2')
    w_s3 = init_weights([num_hidden, num_labels], 'w_s3')
    b_s3 = init_biases([num_labels], 'b_s3')
    w_s4 = init_weights([num_hidden, num_labels], 'w_s4')
    b_s4 = init_biases([num_labels], 'b_s4')
    w_s5 = init_weights([num_hidden, num_labels], 'w_s5')
    b_s5 = init_biases([num_labels], 'b_s5')

    def model(data, keep_prob, shape):
        with tf.name_scope("conv_layer_1"):
            conv_1 = tf.nn.conv2d(data, w_c1, strides=[1, 1, 1, 1], padding='VALID')
            hidden_conv_1 = tf.nn.relu(conv_1 + b_c1)
            pool_1 = tf.nn.max_pool(hidden_conv_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        with tf.name_scope("conv_layer_2"):
            conv_2 = tf.nn.conv2d(pool_1, w_c2, strides=[1, 1, 1, 1], padding='VALID')
            hidden_conv_2 = tf.nn.relu(conv_2 + b_c2)
            pool_2 = tf.nn.max_pool(hidden_conv_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        with tf.name_scope("conv_layer_3"):
            conv_3 = tf.nn.conv2d(pool_2, w_c3, strides=[1, 1, 1, 1], padding='VALID')
            hidden_conv_3 = tf.nn.relu(conv_3 + b_c3)
        with tf.name_scope("fc_layer_1"):
            hidden_drop = tf.nn.dropout(hidden_conv_3, keep_prob)
            shape = hidden_drop.get_shape().as_list()
            reshape = tf.reshape(hidden_drop, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden_fc = tf.nn.relu(tf.matmul(reshape, w_fc1) + b_fc1)
        with tf.name_scope("softmax_1"):
            logits_1 = tf.matmul(hidden_fc, w_s1) + b_s1
        with tf.name_scope("softmax_2"):
            logits_2 = tf.matmul(hidden_fc, w_s2) + b_s2
        with tf.name_scope("softmax_3"):
            logits_3 = tf.matmul(hidden_fc, w_s3) + b_s3
        with tf.name_scope("softmax_4"):
            logits_4 = tf.matmul(hidden_fc, w_s4) + b_s4
        with tf.name_scope("softmax_5"):
            logits_5 = tf.matmul(hidden_fc, w_s5) + b_s5
        return [logits_1, logits_2, logits_3, logits_4, logits_5]

    [logits_1, logits_2, logits_3, logits_4, logits_5] = model(tf_train_dataset, 0.5, shape)
    
    with tf.name_scope("loss"):
        reduce_mean_logits1_digit1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_1, tf_train_labels[:, 1]))
        reduce_mean_logits2_digit2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_2, tf_train_labels[:, 2]))
        reduce_mean_logits3_digit3 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_3, tf_train_labels[:, 3]))
        reduce_mean_logits4_digit4 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_4, tf_train_labels[:, 4]))
        reduce_mean_logits5_digit5 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits_5, tf_train_labels[:, 5]))
        
        loss = reduce_mean_logits1_digit1 + reduce_mean_logits2_digit2 + reduce_mean_logits3_digit3 + reduce_mean_logits4_digit4 + reduce_mean_logits5_digit5

    global_step = tf.Variable(0)
    start_learning_rate = 0.05
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    '''Predictions'''
    def softmax_combine(dataset, shape):
        logit_list = model(dataset, 1.0, shape)
        model_logit_0 = logit_list[0]
        model_logit_1 = logit_list[1]
        model_logit_2 = logit_list[2]
        model_logit_3 = logit_list[3]
        model_logit_4 = logit_list[4]
        train_prediction = tf.pack([
                                    tf.nn.softmax(model_logit_0),
                                    tf.nn.softmax(model_logit_1),
                                    tf.nn.softmax(model_logit_2),
                                    tf.nn.softmax(model_logit_3),
                                    tf.nn.softmax(model_logit_4)])
        return train_prediction

    train_prediction = softmax_combine(tf_train_dataset, shape)
    valid_prediction = softmax_combine(tf_valid_dataset, shape)
    test_prediction = softmax_combine(tf_test_dataset, shape)
    saver = tf.train.Saver()

In [6]:
num_steps = 60000

In [7]:
with tf.Session(graph=graph) as session:
    writer = tf.train.SummaryWriter("log_trial_3", session.graph)  # for 0.8
    merged = tf.merge_all_summaries()
    # saver.restore(session, "model_trial_1.ckpt")
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :, :, :]
        batch_labels = y_train[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data,tf_train_labels: batch_labels}
        _, l, predictions, summary = session.run([optimizer, loss, train_prediction, merged],feed_dict=feed_dict)
        if (step % 500 == 0):
            print(('Minibatch loss at step {}: {}').format(step, l))
            print(('Minibatch accuracy: {}%'.format(accuracy(predictions, batch_labels[:,1:6]))))
            print(('Validation accuracy: {}%'.format(accuracy(valid_prediction.eval(),y_val[:,1:6]))))
    print(('Test accuracy: {}%'.format(accuracy(test_prediction.eval(), y_test[:,1:6]))))
    save_path = saver.save(session, "model_trial_1.ckpt")

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


TypeError: Fetch argument None has invalid type <type 'NoneType'>